In [3]:
import torch
from pathlib import Path
from monai.utils        import set_determinism  
from split_data         import split_data
from transforms         import get_transforms
from model              import base_MTLResidualAttention3DUnet, MTLResidualAttention3DUnet
from train_model        import train_model
from test_model         import test_model
from train_model_base   import train_model_base
from test_model_base    import test_model_base

# Choose whether to train and/or test model(s)
TRAIN           = 0
TEST            = 1

# Choose which models to test
BASE_CASE       = 0
AUX_SEGMENT_3   = 1
AUX_SEGMENT_6   = 1
AUX_RECONSTRUCT = 1

# Parameters
params = {
    'BATCH_SIZE':       2,
    'MAX_EPOCHS':       100,
    'VAL_INT':          10,
    'PRINT_INT':        10
}

# Set deterministic training for reproducibility
set_determinism(seed = 2056)

# Path to data
img_path = Path("../data")
train_files, val_files, test_files = split_data(img_path, 20)

# Create transforms for training
train_transforms, val_transforms, pred_main, label_main, pred_aux_3, label_aux_3, pred_aux_6, label_aux_6 = get_transforms()

# Use CUDA
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Define organ names in the segmentation task
all_organs =  ["Background", "Bladder", "Bone", "Obturator internus", "Transition zone", "Central gland", "Rectum", "Seminal vesicle", "Neurovascular bundle"]
organs = {
    'all': all_organs,
    'main': ["Transition zone", "Central gland"],
    'dict': {organ: float(idx) for idx, organ in enumerate(all_organs)}
    }

----------------------------------------
Splitting data into train-validate-test sets...
The file does not exist
The file does not exist
Images have been divided into train-validate-test sets.
Total number of images:  585
Number of images train-validate-test:  23 - 2 - 4
----------------------------------------
Creating transformations...
Transforms have been defined.


In [4]:
############# BASE CASE #############
if BASE_CASE:
    organs['aux']  = []
    params['TASK'] = 'BASE_CASE'
    model_name     = 'base_case'
    model  = base_MTLResidualAttention3DUnet(in_channels = 1, main_out_channels = len(organs['main'])+1, device=device, gated_attention = True).to(device) 

    if TRAIN:
        torch.cuda.empty_cache()
        train_model_base(model, device, params, train_files, train_transforms, val_files, val_transforms, organs, pred_main, label_main, model_name)
    if TEST:
        torch.cuda.empty_cache()
        test_model_base(model, device, params, test_files, val_transforms, organs, pred_main, label_main, model_name)


############# AUXILIARY TASK - SEGMENT 3 EXTRA STRUCTURES #############
if AUX_SEGMENT_3:
    organs['aux']  = ["Rectum", "Seminal vesicle", "Neurovascular bundle"]
    params['TASK'] = 'SEGMENT'
    model_name     = 'auxiliary_segment_3'
    model = MTLResidualAttention3DUnet(in_channels = 1, main_out_channels = len(organs['main'])+1, aux_out_channels = len(organs['aux'])+1, device=device, gated_attention = True).to(device) 
    
    if TRAIN:
        torch.cuda.empty_cache()
        train_model(model, device, params, train_files, train_transforms, val_files, val_transforms, organs, pred_main, label_main, pred_aux_3, label_aux_3, model_name)
    if TEST:
        torch.cuda.empty_cache()
        test_model(model, device, params, test_files, val_transforms, organs, pred_main, label_main, pred_aux_3, label_aux_3, model_name)
        
        
############# AUXILIARY TASK - SEGMENT 6 EXTRA STRUCTURES #############
if AUX_SEGMENT_6:
    organs['aux']  = ["Rectum", "Seminal vesicle", "Neurovascular bundle", "Bladder", "Bone", "Obturator internus"]
    params['TASK'] = 'SEGMENT'
    model_name     = 'auxiliary_segment_6'
    model = MTLResidualAttention3DUnet(in_channels = 1, main_out_channels = len(organs['main'])+1, aux_out_channels = len(organs['aux'])+1, device=device, gated_attention = True).to(device) 
    
    if TRAIN:
        torch.cuda.empty_cache()
        train_model(model, device, params, train_files, train_transforms, val_files, val_transforms, organs, pred_main, label_main, pred_aux_6, label_aux_6, model_name)
    if TEST:
        torch.cuda.empty_cache()
        test_model(model, device, params, test_files, val_transforms, organs, pred_main, label_main, pred_aux_6, label_aux_6, model_name)
    
    
############# AUXILIARY TASK - RECONSTRUCTION #############
if AUX_RECONSTRUCT:
    organs['aux']   = []
    params['TASK'] = 'RECONSTRUCT'
    model_name     = 'auxiliary_reconstruct'
    model = MTLResidualAttention3DUnet(in_channels = 1, main_out_channels = len(organs['main'])+1, aux_out_channels=1, device=device, gated_attention = True).to(device) 
    if TRAIN:
        torch.cuda.empty_cache()
        train_model(model, device, params, train_files, train_transforms, val_files, val_transforms, organs, pred_main, label_main, pred_aux_3, label_aux_3, model_name)
    if TEST:
        torch.cuda.empty_cache()
        test_model(model, device, params, test_files, val_transforms, organs, pred_main, label_main, pred_aux_3, label_aux_3, model_name)
    


----------------------------------------
Starting model testing...


pixdim[0] (qfac) should be 1 (default) or -1; setting qfac to 1


2023-05-05 12:40:44,891 - pixdim[0] (qfac) should be 1 (default) or -1; setting qfac to 1


/opt/anaconda3/lib/python3.9/site-packages/torch/_tensor.py:1295: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  ret = func(*args, **kwargs)
/opt/anaconda3/lib/python3.9/site-packages/monai/data/__init__.py:127: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  if storage.is_cuda:
/opt/anaconda3/lib/python3.9/site-packages/monai/data/__init__.py:120: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are usi


Mean dice for main task: 0.8630
Mean metric for aux task: 0.0000
----------------------------------------
Starting model testing...


...


Mean dice for main task: 0.8763
Mean metric for aux task: 0.7725
----------------------------------------
Starting model testing...


pixdim[0] (qfac) should be 1 (default) or -1; setting qfac to 1


2023-05-05 12:50:27,798 - pixdim[0] (qfac) should be 1 (default) or -1; setting qfac to 1


/opt/anaconda3/lib/python3.9/site-packages/torch/_tensor.py:1295: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  ret = func(*args, **kwargs)
/opt/anaconda3/lib/python3.9/site-packages/monai/data/__init__.py:127: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  if storage.is_cuda:
/opt/anaconda3/lib/python3.9/site-packages/torch/_tensor.py:1295: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using st


Mean dice for main task: 0.8634
Mean metric for aux task: 0.0003
